<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_03_Consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Banco de Dados Exemplo - Chinook.db (SQLite3)

In [1]:
!wget "https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/chinook.db"

--2021-09-13 20:45:54--  https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/chinook.db
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/valerio-unifei/ECAA07/main/Bancos/chinook.db [following]
--2021-09-13 20:45:55--  https://raw.githubusercontent.com/valerio-unifei/ECAA07/main/Bancos/chinook.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 884736 (864K) [application/octet-stream]
Saving to: ‘chinook.db’

chinook.db          100%[===================>] 864.00K  --.-KB/s    in 0.04s   

2021-09-13 20:45:55 (21.9 MB/s) - ‘chinook.db’ saved [884736/884736]



# Estrutura do Banco

<img src="https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg">

# SQLite3 - Consultas

## Conexão com o Banco de Dados

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chinook.db')

# Exercícios

## Consulta 1

In [12]:
tabelas = pd.read_sql_query(sql='SELECT * FROM sqlite_master WHERE type="table"',con=conn)
tabelasSe 

,type,name,tbl_name,rootpage,sql
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


## Consulta 2

In [4]:
consulta = pd.read_sql_query(sql='SELECT * FROM customers',con=conn)
print('Registros =',len(consulta))
consulta.head()

Registros = 59


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


## Consulta 3

In [14]:
consulta = pd.read_sql_query(sql='SELECT * FROM customers WHERE Fax !="None" ORDER BY FirstName, LastName DESC',con=conn)
print('Registros =',len(consulta))
consulta.head()

Registros = 12


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,11,Alexandre,Rocha,Banco do Brasil S.A.,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,+55 (11) 3055-3278,+55 (11) 3055-8131,alero@uol.com.br,5
1,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4
2,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
3,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


## Consulta 4

In [7]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          STRFTIME('%d/%m/%Y',y.InvoiceDate) as data_compra
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
      """,
      con = conn)
print('Registros =',len(consulta))
consulta.head(20)

Registros = 412


,nome,pais,data_compra
0,Luís Gonçalves,Brazil,11/03/2010
1,Luís Gonçalves,Brazil,13/06/2010
2,Luís Gonçalves,Brazil,15/09/2010
3,Luís Gonçalves,Brazil,06/05/2011
4,Luís Gonçalves,Brazil,27/10/2012
5,Luís Gonçalves,Brazil,07/12/2012
6,Luís Gonçalves,Brazil,07/08/2013
7,Leonie Köhler,Germany,01/01/2009
8,Leonie Köhler,Germany,11/02/2009
9,Leonie Köhler,Germany,12/10/2009


## Consulta 5

In [19]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          STRFTIME('%d/%m/%Y',y.InvoiceDate) as data_compra,
          z.UnitPrice as preco_unitario,
          z.Quantity as quantidade
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
      ORDER BY x.Country, y.InvoiceDate DESC
      """,
      con = conn)
print('Registros =',len(consulta))
consulta.head(20)

Registros = 2240


,nome,pais,data_compra,preco_unitario,quantidade
0,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
1,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
2,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
3,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
4,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
5,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
6,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
7,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
8,Diego Gutiérrez,Argentina,08/11/2013,0.99,1
9,Diego Gutiérrez,Argentina,10/03/2013,0.99,1


## Consulta 6

In [28]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Firstname || ' ' || x.LastName as nome,
          x.Country as pais,
          STRFTIME('%d/%m/%Y',y.InvoiceDate) as data_compra,
          z.UnitPrice as preco_unitario,
          z.Quantity as quantidade,
          t.Name as faixa_nome,
          a.Title as album_nome,
          b.Name as artista,
          m.Name as formato_media
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
        INNER JOIN tracks as t ON z.TrackId = t.TrackId
        INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
    WHERE b.Name LIKE '%&%' and x.Country = 'Canada'
    ORDER BY data_compra DESC, nome
      """,
      con = conn)
print('Registros =',len(consulta))
consulta.head(20)

Registros = 19


,nome,pais,data_compra,preco_unitario,quantidade,faixa_nome,album_nome,artista,formato_media
0,François Tremblay,Canada,30/01/2013,0.99,1,Pot-Pourri N.º 4,Vinícius De Moraes - Sem Limite,Toquinho & Vinícius,MPEG audio file
1,François Tremblay,Canada,30/01/2013,0.99,1,Pot-Pourri N.º 5,Vinícius De Moraes - Sem Limite,Toquinho & Vinícius,MPEG audio file
2,François Tremblay,Canada,30/01/2013,0.99,1,Por Que Será,Vinícius De Moraes - Sem Limite,Toquinho & Vinícius,MPEG audio file
3,François Tremblay,Canada,30/01/2013,0.99,1,Samba Em Prelúdio,Vinícius De Moraes - Sem Limite,Toquinho & Vinícius,MPEG audio file
4,François Tremblay,Canada,28/10/2012,0.99,1,"Concerto for Cello and Orchestra in E minor, O...",Elgar: Cello Concerto & Vaughan Williams: Fant...,"Felix Schmidt, London Symphony Orchestra & Raf...",Protected AAC audio file
5,François Tremblay,Canada,28/10/2012,0.99,1,"Wellington's Victory or the Battle Symphony, O...","Tchaikovsky: 1812 Festival Overture, Op.49, Ca...",Antal Doráti & London Symphony Orchestra,Protected AAC audio file
6,François Tremblay,Canada,28/10/2012,0.99,1,Romeo et Juliette: No. 11 - Danse des Chevaliers,Prokofiev: Romeo & Juliet,Michael Tilson Thomas & San Francisco Symphony,Protected AAC audio file
7,François Tremblay,Canada,28/10/2012,0.99,1,"Symphonie Fantastique, Op. 14: V. Songe d'une ...",Berlioz: Symphonie Fantastique,Michael Tilson Thomas & San Francisco Symphony,Protected AAC audio file
8,Aaron Mitchell,Canada,27/06/2012,0.99,1,Tightrope,In Step,Stevie Ray Vaughan & Double Trouble,MPEG audio file
9,Aaron Mitchell,Canada,27/06/2012,0.99,1,Wall Of Denial,In Step,Stevie Ray Vaughan & Double Trouble,MPEG audio file


## Consulta 7

In [29]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          x.Country as pais,
          COUNT(*) as num_zeca
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
        INNER JOIN tracks as t ON z.TrackId = t.TrackId
        INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      WHERE B.Name = 'Zeca Pagodinho'
      GROUP BY x.Country
      """,
      con = conn)
consulta.head(10)

,pais,num_zeca
0,Czech Republic,2
1,Finland,2
2,Hungary,2
3,USA,3


## Consulta 8

In [35]:
consulta = pd.read_sql_query(
    sql= """
      SELECT
          a.Title as album_nome,
          b.Name as artista,
          COUNT(*) as num_faixas
      FROM tracks as t INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      GROUP BY a.Title, b.Name
      ORDER BY num_faixas DESC
      LIMIT 10
      """,
      con = conn)
consulta

,album_nome,artista,num_faixas
0,Greatest Hits,Lenny Kravitz,57
1,Minha Historia,Chico Buarque,34
2,Unplugged,Eric Clapton,30
3,"Lost, Season 3",Lost,26
4,"Lost, Season 1",Lost,25
5,"The Office, Season 3",The Office,25
6,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),24
7,"Lost, Season 2",Lost,24
8,My Way: The Best Of Frank Sinatra [Disc 1],Frank Sinatra,24
9,Afrociberdelia,Chico Science & Nação Zumbi,23


## Consulta 9

In [39]:
consulta = pd.read_sql_query(
    sql= """
      SELECT 
          a.Title as album_nome,
          b.Name as artista,
          SUM(z.UnitPrice * z.Quantity) as valor,
          COUNT(*) as quantidade
      FROM customers as x INNER JOIN invoices as y ON x.CustomerId = y.CustomerId
        INNER JOIN invoice_items as z ON y.InvoiceID = z.InvoiceID
        INNER JOIN tracks as t ON z.TrackId = t.TrackId
        INNER JOIN albums as a ON t.AlbumId = a.AlbumId
        INNER JOIN artists as b ON a.ArtistId = b.ArtistId
        INNER JOIN media_types as m ON t.MediaTypeId = m.MediaTypeId
      GROUP BY a.Title, b.Name
      ORDER BY valor desc
      LIMIT 10
      """,
      con = conn)
consulta

,album_nome,artista,valor,quantidade
0,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),35.82,18
1,"The Office, Season 3",The Office,31.84,16
2,Minha Historia,Chico Buarque,26.73,27
3,"Heroes, Season 1",Heroes,25.87,13
4,"Lost, Season 2",Lost,25.87,13
5,Greatest Hits,Lenny Kravitz,25.74,26
6,Unplugged,Eric Clapton,24.75,25
7,"Battlestar Galactica, Season 3",Battlestar Galactica,23.88,12
8,"Lost, Season 3",Lost,21.89,11
9,Acústico,Titãs,21.78,22
